In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import itertools
import copy
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)

In [2]:
dataset_name = "/Book"

In [3]:
pwd

'/Users/tianhui/Documents/project/AAAI23/codes/data/prepocess'

# 1. Read Data

In [4]:
item_df = pd.read_csv('../Dataset/Amazon_Books/Amazon_Books.item',sep="\t",encoding="latin-1")
item_df.columns = ["itemid","title","categories","brand","sales_type","sales_rank",]#item_df.columns
item_df.shape[0]
item_df = item_df[item_df["sales_type"]=="Books"].reset_index(drop=True)
item_df
(item_df.isna().sum()/item_df.shape[0]).sort_values(ascending=False)

3023135

,itemid,title,categories,brand,sales_type,sales_rank
0,0000092878,Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series),NaN,Keith Graham,Books,1349781.0
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Self-Assessment Program,'Books',Acp,Books,1702625.0
2,0000004545,"Flex! Discography of North American Punk, Hardcore, and Powerpop 1975-1985 A-M","'Books', 'Music'",Burkhard Jarisch,Books,6291012.0
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal,"'Books', 'Music'",Stamps/Baxter,Books,2384057.0
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple Regular Suede Platforms Shoes,NaN,NaN,Books,11735726.0
...,...,...,...,...,...,...
2824438,B01HJA1TCG,Cool Japanese Oriental Cherry Red Fish Scale Throw 18*18 pillow Case,NaN,NaN,Books,11466168.0
2824439,B01HJA4AYA,"JanCalm LG Optimus Zone 3 Case,LG K4 Case,[Shock Absorption] Dual Layer Protective Case Cover fo...",NaN,NaN,Books,7099614.0
2824440,B01HJA2TAM,"Galaxy Note 5 Case,Luxury Stylish Design Electroplated Slim Fit Lightweight Ultra Thin Metallic ...",NaN,NaN,Books,6041710.0
2824441,B01HJBZV74,Zhanzy Australia Flag Map Men's Women's Unisex Custom Knee High OTC Cushion Athletic Crew Footba...,NaN,NaN,Books,12981948.0


categories    0.131647
brand         0.034287
sales_rank    0.000238
title         0.000064
itemid        0.000000
sales_type    0.000000
dtype: float64

In [5]:
item_df.drop(["categories","brand","sales_type","sales_rank"], axis=1, inplace=True)#"price"
item_df = item_df.dropna().reset_index(drop=True) # fliter no title
item_df

,itemid,title
0,0000092878,Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series)
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Self-Assessment Program
2,0000004545,"Flex! Discography of North American Punk, Hardcore, and Powerpop 1975-1985 A-M"
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple Regular Suede Platforms Shoes
...,...,...
2824258,B01HJA1TCG,Cool Japanese Oriental Cherry Red Fish Scale Throw 18*18 pillow Case
2824259,B01HJA4AYA,"JanCalm LG Optimus Zone 3 Case,LG K4 Case,[Shock Absorption] Dual Layer Protective Case Cover fo..."
2824260,B01HJA2TAM,"Galaxy Note 5 Case,Luxury Stylish Design Electroplated Slim Fit Lightweight Ultra Thin Metallic ..."
2824261,B01HJBZV74,Zhanzy Australia Flag Map Men's Women's Unisex Custom Knee High OTC Cushion Athletic Crew Footba...


In [6]:
rating_df = pd.read_csv('../Dataset/Amazon_Books/Amazon_Books.inter',sep="\t")

In [7]:
rating_df.columns

Index(['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float'], dtype='object')

In [8]:
rating_df.columns = ["userid","itemid","rating","timestamp"]
rating_df = rating_df[rating_df["itemid"].isin(item_df.itemid.tolist())].reset_index(drop=True) # fliter no title
rating_df
(rating_df.isna().sum()/rating_df.shape[0]).sort_values(ascending=False)
rating_df["userid"].nunique()
rating_df["itemid"].nunique()

,userid,itemid,rating,timestamp
0,A1C6M8LCIX4M6M,0001713353,5.0,1123804800
1,A1REUF3A1YCPHM,0001713353,5.0,1112140800
2,A1YRBRK2XM5D5,0001713353,5.0,1081036800
3,A1V8ZR5P78P4ZU,0001713353,5.0,1077321600
4,A2ZB06582NXCIV,0001713353,5.0,1475452800
...,...,...,...,...
50267103,A1ACVVRXKHGNVV,B01HJA2TAM,5.0,1487548800
50267104,A235GU4TJQNAN8,B01HJA4AYA,4.0,1468886400
50267105,A3UDGFA9FBZ3LX,B01HJA1TCG,1.0,1472774400
50267106,A2D2545LW6NUSE,B01HJBZV74,1.0,1480032000


userid       0.0
itemid       0.0
rating       0.0
timestamp    0.0
dtype: float64

15246826

2824117

In [9]:
item_interactions_count = rating_df[["userid","itemid"]].groupby(["itemid"]).count()
item_interactions_count.columns = ["count"]
item_interactions_count.sort_values(["count"] , inplace=True, ascending=False)
item_interactions_count

,count
itemid,
038568231X,58150
0297859382,44956
0007420412,44381
0141353678,37783
0312577222,36620
...,...
0864926634,1
1629114294,1
1629114340,1


In [10]:
item_interactions_count[:10000]

,count
itemid,
038568231X,58150
0297859382,44956
0007420412,44381
0141353678,37783
0312577222,36620
...,...
1596067691,513
0615983987,513
097048190X,513


# 2. Fliter

In [11]:
item_title_exist_idset = item_interactions_count[:10000].index.tolist()
rating_df_fliter = rating_df[rating_df["itemid"].isin(item_title_exist_idset)].reset_index(drop=True)
rating_df_fliter

,userid,itemid,rating,timestamp
0,A310S8GKL783R8,0001384198,5.0,1523577600
1,ABO3HOSXIPOZY,0001384198,4.0,1523404800
2,AU4Z1Q4AX9ILA,0001384198,5.0,1523318400
3,A1TGZVJZK8SMBX,0001384198,5.0,1523232000
4,A3CCP99A7Z7HHT,0001384198,3.0,1523145600
...,...,...,...,...
13325020,A242J1R5A6KVQ1,B01FMVUECK,5.0,1534896000
13325021,A3NX8X5TD6KQ38,B01FMVUECK,4.0,1534896000
13325022,A1QYAHXRRLZS68,B01FMVUECK,3.0,1534377600
13325023,AS4BULNCQKW9S,B01FMVUECK,3.0,1534204800


In [12]:
rating_df_fliter["rating"].value_counts()

rating
5.0    8671144
4.0    2597664
3.0    1040700
1.0     525022
2.0     490492
0.0          3
Name: count, dtype: int64

In [13]:
rating_df_fliter["label"] = rating_df_fliter["rating"].apply(lambda x: 1 if x>=4 else 0)
rating_df_fliter
rating_df_fliter[rating_df_fliter["label"]==1].shape[0] / rating_df_fliter.shape[0]

,userid,itemid,rating,timestamp,label
0,A310S8GKL783R8,0001384198,5.0,1523577600,1
1,ABO3HOSXIPOZY,0001384198,4.0,1523404800,1
2,AU4Z1Q4AX9ILA,0001384198,5.0,1523318400,1
3,A1TGZVJZK8SMBX,0001384198,5.0,1523232000,1
4,A3CCP99A7Z7HHT,0001384198,3.0,1523145600,0
...,...,...,...,...,...
13325020,A242J1R5A6KVQ1,B01FMVUECK,5.0,1534896000,1
13325021,A3NX8X5TD6KQ38,B01FMVUECK,4.0,1534896000,1
13325022,A1QYAHXRRLZS68,B01FMVUECK,3.0,1534377600,0
13325023,AS4BULNCQKW9S,B01FMVUECK,3.0,1534204800,0


0.8456875690664746

In [14]:
rating_df_fliter = rating_df_fliter.sort_values('rating', ascending=False)

In [15]:
item_pop = rating_df_fliter[rating_df_fliter["label"]==1]["itemid"].value_counts().rename_axis("itemid").reset_index(name="pop")
item_pop

,itemid,pop
0,038568231X,43434
1,0007420412,38971
2,0141353678,35158
3,0312577222,35072
4,0297859382,31347
...,...,...
9995,1466938897,128
9996,0761919945,102
9997,1496018958,84
9998,0399147659,70


In [16]:
rating_df_fliter.sort_values(["userid", "timestamp"] , inplace=True, ascending=True) 
rating_df_fliter = rating_df_fliter.reset_index(drop=True)
sequence_df = rating_df_fliter.groupby(['userid']).agg(
    itemid_seq=("itemid", list),
    label_seq=("label", list)
).reset_index()
sequence_df

,userid,itemid_seq,label_seq
0,A0001392IVCRENBEIEYS,[0062224743],[0]
1,A0002032ZFQKDVHYKGWR,[0451211634],[1]
2,A0002090WKEMAO8KOWKM,[1440309310],[1]
3,A0002802PGRRB05CR0VT,[0061251348],[0]
4,A00032921HLX2KJJVXRS,[0529120887],[1]
...,...,...,...
5851579,AZZZSEW586KLX,[038534905X],[1]
5851580,AZZZTAPYKI9RD,[0764210890],[1]
5851581,AZZZWEMB4N2W1,"[0385346859, 1910751774, 0143109464, 0143110381]","[0, 0, 0, 0]"
5851582,AZZZWHX2LWVBS,[0143108867],[1]


In [17]:
def get_seq(row):
    pos_seq = []
    neg_seq = []
    for i in range(len(row.label_seq)):
        if row.label_seq[i]==1:
            pos_seq.append(row.itemid_seq[i])
        else:
            neg_seq.append(row.itemid_seq[i])
    return pos_seq, neg_seq, len(row.itemid_seq), len(pos_seq), len(neg_seq)

sequence_df[["pos_seq","neg_seq", "seq_length", "pos_seq_length", "neg_seq_length"]] = sequence_df.apply(get_seq, axis=1, result_type="expand") 
sequence_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,A0001392IVCRENBEIEYS,[0062224743],[0],[],[0062224743],1,0,1
1,A0002032ZFQKDVHYKGWR,[0451211634],[1],[0451211634],[],1,1,0
2,A0002090WKEMAO8KOWKM,[1440309310],[1],[1440309310],[],1,1,0
3,A0002802PGRRB05CR0VT,[0061251348],[0],[],[0061251348],1,0,1
4,A00032921HLX2KJJVXRS,[0529120887],[1],[0529120887],[],1,1,0
...,...,...,...,...,...,...,...,...
5851579,AZZZSEW586KLX,[038534905X],[1],[038534905X],[],1,1,0
5851580,AZZZTAPYKI9RD,[0764210890],[1],[0764210890],[],1,1,0
5851581,AZZZWEMB4N2W1,"[0385346859, 1910751774, 0143109464, 0143110381]","[0, 0, 0, 0]",[],"[0385346859, 1910751774, 0143109464, 0143110381]",4,0,4
5851582,AZZZWHX2LWVBS,[0143108867],[1],[0143108867],[],1,1,0


In [18]:
sequence_df[(sequence_df["pos_seq_length"]>=6) & (sequence_df["neg_seq_length"]>=6)].shape[0]
sequence_df[(sequence_df["pos_seq_length"]>=12) & (sequence_df["neg_seq_length"]>=10)].shape[0]
sequence_df[(sequence_df["pos_seq_length"]>=13) & (sequence_df["neg_seq_length"]>=10)].shape[0]

22577

7474

7216

In [19]:
sequence_df[(sequence_df["pos_seq_length"]>=13) & (sequence_df["neg_seq_length"]>=20)].shape[0]

2224

In [20]:
sequence_df_fliter = sequence_df[(sequence_df["pos_seq_length"]>=13) & (sequence_df["neg_seq_length"]>=10)].reset_index(drop=True)
item_df = item_df[item_df["itemid"].isin(list(itertools.chain.from_iterable(sequence_df_fliter["itemid_seq"].values.tolist())))].reset_index(drop=True)
sequence_df_fliter
item_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,A101YS38SKOZIR,"[0141027126, 143916469X, 0307949702, 0062248472, 0980017106, 1612187218, 0297859382, 0552778478,...","[1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,...","[0141027126, 0980017106, 1612187218, 0552778478, 0399159347, 0778315339, 0375969020, 0141378247,...","[143916469X, 0307949702, 0062248472, 0297859382, 1475043740, 0316055433, 1410466264, 1477818421,...",73,51,22
1,A102Z3T7NSM5KC,"[0316909718, 0307387941, 0739328174, 0307970213, 0613171373, 0449205460, 0307278441, 0140187642,...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,...","[0316909718, 0307387941, 0739328174, 0307970213, 0449205460, 0307278441, 0140187642, 1616200464,...","[0613171373, 0307941515, 0062060627, 0964326108, 1414319363, 0751541184, 0060855924, 0316666009,...",116,99,17
2,A103KWX1QFCG6I,"[0441019730, 0425256960, 0800757181, 0312383355, 0843957352, 044102002X, 0978924622, 0226104206,...","[1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[0441019730, 0843957352, 044102002X, 0978924622, 0226104206, 044654759X, 1612185932, 0349407061,...","[0425256960, 0800757181, 0312383355, 1490439838, 0425267040, 0425273865, 1480568112, 0345544013,...",71,59,12
3,A103M7E0BSFC83,"[1469290103, 1461130174, 1492722634, 1494928655, 0983887918, 1490536388, 0399159347, 1495472418,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,...","[1469290103, 1461130174, 1492722634, 1494928655, 0983887918, 1490536388, 0399159347, 1495472418,...","[1476786402, 1477824952, 1477827757, 0099543680, 0143108867, 0224102362, 1491277289, 0593075307,...",90,76,14
4,A1047EDJ84IMAS,"[077830289X, 0385341008, 0802402119, 0307265439, 1482320274, 1401940676, 1463717091, 0615659292,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,...","[077830289X, 0385341008, 0802402119, 1482320274, 1401940676, 1463717091, 0615659292, B004QK6Z6I,...","[0307265439, 0983920877, 0749958790, 1493645935, 0307956350, 0615698883, 0552774987, 1475007671,...",60,47,13
...,...,...,...,...,...,...,...,...
7211,AZY0ZLZHJYTSE,"[0515131679, B006VPAJ54, 0143144014, 0987741101, 1493547658, 1937120341, 1480183679, 1556114931,...","[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1]","[0515131679, 1937120341, 1480183679, 1556114931, 1105086380, 1480219851, 1484054172, 1484823664,...","[B006VPAJ54, 0143144014, 0987741101, 1493547658, 1469217090, 1481882902, 1494430533, 1477809317,...",25,15,10
7212,AZY96LU53VWGO,"[0451216768, 0739471716, 0778314154, 0778304175, 0976876051, 0451225090, 0800734211, 0345450736,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,...","[0451216768, 0739471716, 0778314154, 0976876051, 0451225090, 0800734211, 0345450736, 0345441133,...","[0778304175, 1602857679, 055216402X, 1480573876, 1612185894, 0310315476, 0764204807, 0345541790,...",87,73,14
7213,AZYERRDY2VW61,"[150287721X, 1544115202, 147003655X, 0989416410, 0989776018, 1482323974, 1492788678, 0989311287,...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[150287721X, 1544115202, 1481859242, 1480054542, 1484966287, 0800719980, 0985777338, 0373077165,...","[147003655X, 0989416410, 0989776018, 1482323974, 1492788678, 0989311287, 1484968042, 1480238902,...",47,28,19
7214,AZZIFY29OWS6B,"[1456527738, 1467940151, 144235948X, 146109111X, 1455599050, 1481268503, 0991686004, 147674355X,...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[1467940151, 144235948X, 1455599050, 1481268503, 0991686004, 147674355X, 0615741932, 0615752179,...","[1456527738, 146109111X, 1471118630, 0988301318, 1419970291, 1939962315, 098870742X, 1480114480,...",25,14,11


,itemid,title
0,0001050230,"Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan &amp; Cast"
1,000100039X,The Prophet
2,0001048767,Othello: Complete &amp; Unabridged
3,0001384198,The Little Engine That Could
4,0001046314,A Woman of Substance
...,...,...
9812,B017QEJSIW,The Essential Life (2nd Edition)
9813,B0197P8UQ4,By Yuval Noah Harari - Sapiens: A Brief History of Humankind (MP3 - Unabridged CD) (2015-02-25) ...
9814,B01B76KBR2,Colorful Blessings: Cards to Color and Share
9815,B01FGM4H4Q,All Quiet on the Western Front (World War I) by Erich Maria Remarque (2014-03-15)


# 3. LabelEncoder

In [21]:
# LabelEncoder 
from sklearn import preprocessing

user_le = preprocessing.LabelEncoder()
user_le.fit(sequence_df_fliter['userid'].values.tolist())
print("user id unique nums:", len(user_le.classes_))
sequence_df_fliter['userid'] = pd.Series(user_le.transform(sequence_df_fliter['userid']))


item_le = preprocessing.LabelEncoder()
item_le.fit(list(itertools.chain.from_iterable(sequence_df_fliter["itemid_seq"].values.tolist())))
print("item id unique nums:", len(item_le.classes_))
sequence_df_fliter['pos_seq'] = sequence_df_fliter['pos_seq'].apply(lambda x: list(item_le.transform(x)))
sequence_df_fliter['neg_seq'] = sequence_df_fliter['neg_seq'].apply(lambda x: list(item_le.transform(x)))

sequence_df_fliter

LabelEncoder()

user id unique nums: 7216


LabelEncoder()

item id unique nums: 9817


,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,0,"[0141027126, 143916469X, 0307949702, 0062248472, 0980017106, 1612187218, 0297859382, 0552778478,...","[1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,...","[1272, 5552, 9155, 4061, 3313, 5009, 3023, 1358, 2301, 7886, 1349, 7230, 615, 5248, 6483, 5295, ...","[6487, 1999, 749, 1729, 7095, 2326, 6328, 7276, 118, 5008, 218, 2772, 826, 7388, 5233, 1830, 279...",73,51,22
1,1,"[0316909718, 0307387941, 0739328174, 0307970213, 0613171373, 0449205460, 0307278441, 0140187642,...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,...","[2533, 1821, 4632, 2020, 3727, 1782, 1165, 9168, 2529, 1931, 3036, 2643, 4072, 491, 1999, 86, 71...","[4254, 1988, 656, 5484, 6356, 4779, 495, 2519, 4058, 1059, 1989, 2979, 1421, 3115, 1771, 2040, 2...",116,99,17
2,2,"[0441019730, 0425256960, 0800757181, 0312383355, 0843957352, 044102002X, 0978924622, 0226104206,...","[1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[3617, 5353, 3618, 5541, 1655, 3674, 9150, 2840, 6821, 6829, 2876, 2923, 8063, 3530, 5868, 6102,...","[3489, 5194, 2186, 7772, 3513, 3534, 7591, 2775, 4298, 8101, 5890, 7732]",71,59,12
3,3,"[1469290103, 1461130174, 1492722634, 1494928655, 0983887918, 1490536388, 0399159347, 1495472418,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,...","[7029, 6905, 7864, 7954, 5622, 7782, 3313, 7972, 7781, 7993, 7276, 5935, 7983, 7386, 7154, 7149,...","[7183, 7354, 7406, 1052, 1463, 1654, 7807, 4202, 4588, 4589, 3998, 2841, 3782, 841]",90,76,14
4,4,"[077830289X, 0385341008, 0802402119, 0307265439, 1482320274, 1401940676, 1463717091, 0615659292,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,...","[4994, 3077, 5214, 7664, 6234, 6934, 4309, 9775, 9723, 9747, 1634, 254, 4921, 6213, 3044, 9376, ...","[1757, 5625, 4771, 7898, 2008, 4314, 4058, 7093, 7672, 8579, 6960, 9385, 4204]",60,47,13
...,...,...,...,...,...,...,...,...
7211,7211,"[0515131679, B006VPAJ54, 0143144014, 0987741101, 1493547658, 1937120341, 1480183679, 1556114931,...","[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1]","[3904, 9440, 7562, 8920, 6040, 7568, 7722, 7734, 7842, 7882, 8004, 1971, 7305, 8554, 3460]","[9781, 1552, 5718, 7893, 7000, 7642, 7927, 7242, 5609, 3452]",25,15,10
7212,7212,"[0451216768, 0739471716, 0778314154, 0778304175, 0976876051, 0451225090, 0800734211, 0345450736,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,...","[3749, 4672, 5001, 5528, 3754, 5191, 2655, 2650, 4674, 3849, 4741, 2220, 1995, 4579, 3314, 4997,...","[4995, 9061, 4054, 7592, 9149, 2073, 4890, 2761, 2772, 7940, 550, 204, 3818, 6844]",87,73,14
7213,7213,"[150287721X, 1544115202, 147003655X, 0989416410, 0989776018, 1482323974, 1492788678, 0989311287,...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[8185, 8891, 7640, 7547, 7744, 5162, 5692, 2898, 9379, 7519, 5684, 5313, 4500, 7799, 5641, 8593,...","[7046, 5819, 5851, 7667, 7872, 5815, 7745, 7573, 4188, 5925, 7737, 6972, 5845, 4302, 7675, 5714,...",47,28,19
7214,7214,"[1456527738, 1467940151, 144235948X, 146109111X, 1455599050, 1481268503, 0991686004, 147674355X,...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[6969, 6547, 6858, 7627, 5927, 7149, 4324, 4326, 4330, 789, 4331, 5819, 794, 6804]","[6884, 6902, 7059, 5743, 6410, 9497, 5770, 7553, 7689, 5799, 5775]",25,14,11


In [22]:
item_pop = item_pop[item_pop["itemid"].isin(list(item_df['itemid'].values.tolist()))].reset_index(drop=True)
item_pop["itemid"] = pd.Series(item_le.transform(item_pop['itemid']))

item_df["itemid"] = pd.Series(item_le.transform(item_df['itemid']))

In [23]:
# item_df.to_csv(f"../Dataset/full_item_df.csv", sep="\t", index=False)

# 4. Gnerate Test Data for Top-K Ranking

In [24]:
def get_new_df(df, n):
    df_new = copy.deepcopy(df)
    df_new["pos_seq"] = df_new["pos_seq"].apply(lambda x: x[:-n]) 
    df_new["pos_seq_length"] = df_new["pos_seq"].apply(lambda x:len(x))

    return df_new

sequence_df_fliter["data_index"] = sequence_df_fliter["userid"].apply(lambda x: 0)
sequence_df_fliter_list = [sequence_df_fliter]
for i in range(1, 3):
    sequence_df_fliter_new = get_new_df(sequence_df_fliter, i)
    sequence_df_fliter_new["data_index"] = sequence_df_fliter_new["userid"].apply(lambda x: i)
    sequence_df_fliter_list.append(sequence_df_fliter_new)
    
full_sequence_df = pd.concat(sequence_df_fliter_list).reset_index(drop=True)
full_sequence_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length,data_index
0,0,"[0141027126, 143916469X, 0307949702, 0062248472, 0980017106, 1612187218, 0297859382, 0552778478,...","[1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,...","[1272, 5552, 9155, 4061, 3313, 5009, 3023, 1358, 2301, 7886, 1349, 7230, 615, 5248, 6483, 5295, ...","[6487, 1999, 749, 1729, 7095, 2326, 6328, 7276, 118, 5008, 218, 2772, 826, 7388, 5233, 1830, 279...",73,51,22,0
1,1,"[0316909718, 0307387941, 0739328174, 0307970213, 0613171373, 0449205460, 0307278441, 0140187642,...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,...","[2533, 1821, 4632, 2020, 3727, 1782, 1165, 9168, 2529, 1931, 3036, 2643, 4072, 491, 1999, 86, 71...","[4254, 1988, 656, 5484, 6356, 4779, 495, 2519, 4058, 1059, 1989, 2979, 1421, 3115, 1771, 2040, 2...",116,99,17,0
2,2,"[0441019730, 0425256960, 0800757181, 0312383355, 0843957352, 044102002X, 0978924622, 0226104206,...","[1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[3617, 5353, 3618, 5541, 1655, 3674, 9150, 2840, 6821, 6829, 2876, 2923, 8063, 3530, 5868, 6102,...","[3489, 5194, 2186, 7772, 3513, 3534, 7591, 2775, 4298, 8101, 5890, 7732]",71,59,12,0
3,3,"[1469290103, 1461130174, 1492722634, 1494928655, 0983887918, 1490536388, 0399159347, 1495472418,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,...","[7029, 6905, 7864, 7954, 5622, 7782, 3313, 7972, 7781, 7993, 7276, 5935, 7983, 7386, 7154, 7149,...","[7183, 7354, 7406, 1052, 1463, 1654, 7807, 4202, 4588, 4589, 3998, 2841, 3782, 841]",90,76,14,0
4,4,"[077830289X, 0385341008, 0802402119, 0307265439, 1482320274, 1401940676, 1463717091, 0615659292,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,...","[4994, 3077, 5214, 7664, 6234, 6934, 4309, 9775, 9723, 9747, 1634, 254, 4921, 6213, 3044, 9376, ...","[1757, 5625, 4771, 7898, 2008, 4314, 4058, 7093, 7672, 8579, 6960, 9385, 4204]",60,47,13,0
...,...,...,...,...,...,...,...,...,...
21643,7211,"[0515131679, B006VPAJ54, 0143144014, 0987741101, 1493547658, 1937120341, 1480183679, 1556114931,...","[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1]","[3904, 9440, 7562, 8920, 6040, 7568, 7722, 7734, 7842, 7882, 8004, 1971, 7305]","[9781, 1552, 5718, 7893, 7000, 7642, 7927, 7242, 5609, 3452]",25,13,10,2
21644,7212,"[0451216768, 0739471716, 0778314154, 0778304175, 0976876051, 0451225090, 0800734211, 0345450736,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,...","[3749, 4672, 5001, 5528, 3754, 5191, 2655, 2650, 4674, 3849, 4741, 2220, 1995, 4579, 3314, 4997,...","[4995, 9061, 4054, 7592, 9149, 2073, 4890, 2761, 2772, 7940, 550, 204, 3818, 6844]",87,71,14,2
21645,7213,"[150287721X, 1544115202, 147003655X, 0989416410, 0989776018, 1482323974, 1492788678, 0989311287,...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[8185, 8891, 7640, 7547, 7744, 5162, 5692, 2898, 9379, 7519, 5684, 5313, 4500, 7799, 5641, 8593,...","[7046, 5819, 5851, 7667, 7872, 5815, 7745, 7573, 4188, 5925, 7737, 6972, 5845, 4302, 7675, 5714,...",47,26,19,2
21646,7214,"[1456527738, 1467940151, 144235948X, 146109111X, 1455599050, 1481268503, 0991686004, 147674355X,...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[6969, 6547, 6858, 7627, 5927, 7149, 4324, 4326, 4330, 789, 4331, 5819]","[6884, 6902, 7059, 5743, 6410, 9497, 5770, 7553, 7689, 5799, 5775]",25,12,11,2


In [25]:
import random
random.seed(2023)

def get_pos_neg_target(row):
    return row.pos_seq[-1], row.neg_seq[-1]


def get_full_candidate(row, n_candidate):
    cache = copy.deepcopy(row.neg_seq)
    cache.remove(row.neg_id_target)
    candidate = [row.pos_id_target] + [row.neg_id_target] + (random.sample(cache, n_candidate-2))
    random.shuffle(candidate)
    
    return candidate


def get_sub_candidate(row, n_candidate):
    cand_cache = copy.deepcopy(row.full_candidate)
    cand_cache.remove(row.pos_id_target)
    cand_cache.remove(row.neg_id_target)
    candidate = [row.pos_id_target] + [row.neg_id_target] + (random.sample(list(cand_cache), n_candidate-2))
    random.shuffle(candidate)
    
    return candidate


def get_full_history(row, n_history):
    history = row.pos_seq[-1-n_history : -1]
    return history


def get_sub_history(row, n_history):
    history = row.full_history[0-n_history : ]
    return history

def get_pos_target_index(row):
    return row["itemid_candidate"].index(row["pos_id_target"])

In [26]:
a=range(15)
print([i for i in a])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [27]:
full_sequence_df=full_sequence_df[(full_sequence_df["pos_seq_length"]>=6) & (full_sequence_df["neg_seq_length"]>=30)]

full_sequence_df_new = full_sequence_df[["userid","pos_seq","neg_seq","data_index"]]
full_sequence_df_new[["pos_id_target", "neg_id_target"]] = full_sequence_df_new.apply(get_pos_neg_target, axis=1, result_type="expand")
full_sequence_df_new["full_candidate"] = full_sequence_df_new.apply(get_full_candidate, n_candidate=30, axis=1)

In [28]:
full_sequence_df_new["full_history"] = full_sequence_df_new.apply(get_full_history, n_history=10, axis=1)
full_sequence_df_new

,userid,pos_seq,neg_seq,data_index,pos_id_target,neg_id_target,full_candidate,full_history
12,12,"[1433, 4084, 1229, 2549, 37, 871, 641, 3523, 3555, 577, 79, 1593, 2478, 8657, 2707, 2017, 4434, ...","[3504, 4772, 2831, 2830, 2829, 4112, 1972, 2776, 1614, 4976, 6614, 851, 2218, 5208, 9010, 5211, ...",0,6030,3437,"[2866, 8991, 3437, 911, 5247, 2831, 3164, 966, 5211, 4131, 4123, 1614, 3926, 4415, 9287, 4113, 6...","[2329, 3111, 1870, 2752, 2753, 2754, 4208, 6159, 4118, 3431]"
21,21,"[6410, 4232, 9117, 6858, 6860, 6860, 4338, 6804, 4339, 4331, 7147, 7131, 4329, 9435, 7169, 9096,...","[6860, 6750, 4329, 6742, 6869, 3499, 1718, 7732, 7522, 3522, 3521, 7157, 2787, 794, 4275, 7872, ...",0,2884,8372,"[7732, 4329, 2884, 3532, 6704, 6860, 4300, 3612, 7872, 3616, 6105, 3487, 8372, 6742, 6869, 728, ...","[783, 2838, 2849, 2866, 3788, 3798, 3799, 3823, 6009, 8756]"
31,31,"[3618, 1226, 1132, 2876, 634, 3506, 5162, 3520, 3495, 3484, 3608, 3607, 3610, 9032, 7092, 2877, ...","[2138, 1047, 2899, 2900, 2878, 9466, 1337, 5358, 6112, 1339, 7576, 3178, 7087, 6166, 9239, 1734,...",0,4898,2704,"[4003, 9076, 4226, 1337, 2900, 1027, 165, 4898, 3823, 386, 4484, 1547, 6112, 10, 399, 3011, 1734...","[942, 1099, 1315, 4183, 1163, 9020, 4814, 1634, 2868, 2884]"
34,34,"[6310, 9670, 3299, 4153, 4267, 4061, 2821, 5262, 9790, 2417, 4220, 6745, 2326, 3782, 2363, 3313,...","[3318, 3096, 226, 3070, 6300, 2411, 9131, 6323, 2418, 1732, 2962, 1428, 4234, 1081, 6590, 3164, ...",0,6039,3434,"[7387, 2962, 4234, 3389, 2503, 4589, 2219, 3164, 4607, 2470, 6323, 1995, 2416, 1081, 226, 3096, ...","[2841, 2834, 899, 3387, 913, 2462, 5389, 78, 5920, 3415]"
38,38,"[7268, 3746, 4061, 2219, 7390, 7393, 7393, 3749, 180, 180, 2839, 9132, 2311, 5725, 188, 2324, 69...","[6810, 6510, 7535, 4059, 7216, 5710, 7218, 7642, 8045, 789, 5570, 9001, 4680, 5487, 8942, 1458, ...",0,7799,8672,"[7612, 2915, 6510, 7533, 5570, 5532, 4984, 8045, 9264, 8672, 6528, 7940, 8611, 7672, 7799, 6969,...","[9506, 266, 5903, 8851, 7979, 8472, 7610, 3041, 5175, 5811]"
...,...,...,...,...,...,...,...,...
21618,7186,"[9726, 9726, 57, 2690, 3196, 6051, 1063, 3813, 2317, 1085, 5262, 9790, 2038, 6924, 1237, 2528, 4...","[9698, 1477, 1989, 2341, 3689, 9217, 7149, 3346, 5857, 7620, 5076, 6052, 2744, 7041, 5791, 692, ...",2,2806,3274,"[6799, 1830, 3346, 5263, 1574, 8579, 2192, 6047, 3347, 694, 2806, 2765, 2752, 7149, 2153, 2999, ...","[1964, 3109, 6339, 1950, 2222, 4197, 4444, 2783, 2428, 5401]"
21619,7187,"[3557, 9680, 1953, 2740, 587, 3136, 3141, 4101, 112, 25, 1780, 3557, 9680, 6606, 1485, 129, 171,...","[127, 193, 200, 26, 3691, 154, 584, 4681, 6532, 4758, 1244, 2027, 2316, 2360, 2238, 1868, 4100, ...",2,3441,9148,"[2316, 1857, 967, 584, 201, 8415, 200, 4681, 784, 2703, 638, 152, 3441, 2360, 4100, 9095, 5301, ...","[4581, 1122, 3430, 4808, 3167, 2782, 3551, 4118, 8318, 2788]"
21632,7200,"[5268, 3725, 202, 4779, 1424, 1362, 666, 625, 4018, 1313, 1323, 1315, 3159, 677, 676, 623, 1058,...","[643, 3910, 2637, 1729, 1728, 1324, 1325, 2348, 1890, 1022, 1112, 6539, 1917, 3582, 188, 1180, 3...",2,2280,804,"[1324, 4151, 2031, 1340, 188, 1593, 586, 4150, 1180, 4776, 2280, 2326, 1112, 1325, 152, 218, 158...","[803, 2238, 1253, 462, 792, 4202, 6177, 6268, 6258, 2292]"
21634,7202,"[2620, 7220, 3508, 7479, 7906, 7173, 4587, 6595, 7146, 7156, 6595, 8087, 7963, 7670, 7870, 8200,...","[9692, 5908, 4363, 6547, 7147, 5894, 4117, 5746, 1556, 6860, 4338, 7268, 8023, 2787, 5002, 7745,...",2,9169,9653,"[9583, 8543, 8163, 8005, 4117, 9169, 6547, 6973, 3776, 8171, 7991, 8439, 3767, 9566, 5885, 8585,...","[6951, 9595, 9634, 9283, 8903, 8035, 8910, 9635, 9556, 5972]"


In [29]:
full_sequence_df_new.to_csv(f"../Dataset/full_sequence_df.csv", sep="\t", index=False)

In [30]:
def get_pairwise_data(row):
    candidate = row.itemid_candidate
    pos = candidate[row.pos_target_index]
    neg_list = copy.deepcopy(candidate)
    neg_list.remove(pos)
    pair_list = []
    answer_list = []
    for neg in neg_list:
        pair = [pos, neg]
        random.shuffle(pair)
        answer = pair.index(pos)
        pair_list.append(pair)
        answer_list.append(answer)
    
    return pair_list, answer_list

In [31]:
def get_sub_candidate_random(row, n_candidate, id2item_dict):
    keys = list(id2item_dict.keys())  # 获取所有的键
    keys.remove(row.pos_id_target)  # 移除pos_id_target
    
    candidate_keys = [row.pos_id_target]  # 将pos_id_target添加到候选者列表中
    candidate_keys += random.sample(keys, n_candidate - 1)  # 随机选择n_candidate - 1个键
    
    random.shuffle(candidate_keys)  # 随机打乱候选者列表
    
    return candidate_keys

In [33]:
import copy
def get_topk_final_data(df,id2item_dict):
    for n_candidate in [5]:
        for n_history in [5]:
            print(f"n_candidate:{n_candidate} ; n_history{n_history}")
            LLM_top1_data = copy.deepcopy(df)
            # LLM_top1_data["itemid_candidate"] = LLM_top1_data.apply(get_sub_candidate, n_candidate=n_candidate, axis=1)
            LLM_top1_data["itemid_candidate"] = LLM_top1_data.apply(get_sub_candidate_random, n_candidate=n_candidate, axis=1,id2item_dict=id2item_dict)
            LLM_top1_data["itemid_history"] = LLM_top1_data.apply(get_sub_history, n_history=n_history, axis=1)
            LLM_top1_data["pos_target_index"] = LLM_top1_data.apply(get_pos_target_index, axis=1)
            LLM_top1_data = LLM_top1_data.sample(frac=1.0, random_state=2023).reset_index(drop=True)
            LLM_top1_data[["pair_itemid_candidate", "pair_pos_target_index"]] = LLM_top1_data.apply(get_pairwise_data, axis=1, result_type="expand")
            LLM_top1_data.to_csv(f"../{dataset_name}/topk_candidate@{n_candidate}_history@{n_history}.csv", sep="\t", index=False)

In [ ]:
import json
def get_datamaps(datamaps_path: str):
    """Get data maps from data path. 

    Args:
        datamaps_path (str): The path where data is.

    Returns:
        Tuple[Dict[int, str], Dict[str, int]]: id2item mapping and item2id mapping.
    """
    with open(datamaps_path, "r") as f:
        datamaps = json.load(f)
    id2item_dict = {int(k): v for k, v in datamaps["id2item_dict"].items()}
    item2id_dict = {k: int(v) for k, v in datamaps["item2id_dict"].items()}

    return id2item_dict, item2id_dict
datamaps_path="../Book/datamaps_cands-30.json"
id2item_dict, item_dict2id = get_datamaps(datamaps_path=datamaps_path)

In [39]:
# -----------------------
# import pandas as pd
# import random
# full_sequence_df_new=pd.read_csv("/code/matianhui/llm_rank/LLM4RS/data/Dataset/full_sequence_df.csv",sep="\t")
# full_sequence_df_new["full_candidate"] = full_sequence_df_new["full_candidate"].apply(eval)
# full_sequence_df_new["full_history"]=full_sequence_df_new["full_history"].apply(eval)
# dataset_name="Book"
# ----------------------
get_topk_final_data(full_sequence_df_new,id2item_dict) # id2item_dict

n_candidate:5 ; n_history5


In [136]:
# full_sequence_df_new["full_candidate"] = full_sequence_df_new["full_candidate"].apply(eval)
# type(full_sequence_df_new.loc[0,"full_candidate"])

In [137]:
full_sequence_df_new.keys()

Index(['userid', 'pos_seq', 'neg_seq', 'data_index', 'pos_id_target',
       'neg_id_target', 'full_candidate', 'full_history'],
      dtype='object')

# 5. Generate Datamaps

In [36]:
def process_name(s):
    s = s.replace("&amp;", "&")
    s = s.replace("  ", " ")
    s = s.replace("\n", " ")
    s = s.replace("\\", " ")
    return s
item_df["title"] = item_df["title"].apply(lambda x: process_name(x))
item_df

,itemid,title
0,3,"Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan & Cast"
1,0,The Prophet
2,2,Othello: Complete & Unabridged
3,5,The Little Engine That Could
4,1,A Woman of Substance
...,...,...
9812,9812,The Essential Life (2nd Edition)
9813,9813,By Yuval Noah Harari - Sapiens: A Brief History of Humankind (MP3 - Unabridged CD) (2015-02-25) ...
9814,9814,Colorful Blessings: Cards to Color and Share
9815,9815,All Quiet on the Western Front (World War I) by Erich Maria Remarque (2014-03-15)


In [37]:
id2item_dict = item_df.set_index("itemid")["title"].to_dict()
id2item_dict
item2id_dict = item_df.set_index("title")["itemid"].to_dict()
item2id_dict

{3: "Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan & Cast",
 0: 'The Prophet',
 2: 'Othello: Complete & Unabridged',
 5: 'The Little Engine That Could',
 1: 'A Woman of Substance',
 4: "A Child's Garden of Verses",
 8: 'Green Eggs and Ham',
 7: 'Lion, the Witch and the Wardrobe',
 6: 'Go, Dog. Go! (Beginner Series)',
 10: "Magician's Nephew - Folio Society Hardcover",
 12: 'The Secret Garden',
 9: 'Fox in Socks (Dr.Seuss Board Books)',
 11: 'Princess and the Goblin (Abridged Classics)',
 14: 'PREY.',
 13: 'Cat in the Hat Song Book',
 15: 'Gilead',
 16: 'The Monk Who Sold His Ferrari: A Remarkable Story About Living Your Dreams',
 17: 'For Whom the Bell Tolls',
 18: 'The Sas Survival Handbook',
 19: 'The One Minute Manager',
 20: 'The Winds of War',
 21: 'North and South',
 23: 'Without Remorse',
 22: 'Kiss The Girls',
 26: 'A Feast for Crows (Song of Ice and Fire)',
 24: 'The Golden Fool (Tawny Man)',
 25: 'A Dance with Dragons',
 27: 'Jack And Jill',
 28: 'The God 

{"Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan & Cast": 3,
 'The Prophet': 993,
 'Othello: Complete & Unabridged': 2,
 'The Little Engine That Could': 5,
 'A Woman of Substance': 1,
 "A Child's Garden of Verses": 4,
 'Green Eggs and Ham': 8,
 'Lion, the Witch and the Wardrobe': 7,
 'Go, Dog. Go! (Beginner Series)': 6,
 "Magician's Nephew - Folio Society Hardcover": 10,
 'The Secret Garden': 9163,
 'Fox in Socks (Dr.Seuss Board Books)': 9,
 'Princess and the Goblin (Abridged Classics)': 11,
 'PREY.': 14,
 'Cat in the Hat Song Book': 13,
 'Gilead': 15,
 'The Monk Who Sold His Ferrari: A Remarkable Story About Living Your Dreams': 16,
 'For Whom the Bell Tolls': 17,
 'The Sas Survival Handbook': 18,
 'The One Minute Manager': 19,
 'The Winds of War': 2535,
 'North and South': 21,
 'Without Remorse': 49,
 'Kiss The Girls': 22,
 'A Feast for Crows (Song of Ice and Fire)': 26,
 'The Golden Fool (Tawny Man)': 24,
 'A Dance with Dragons': 25,
 'Jack And Jill': 27,
 'The Go

In [38]:
datamaps = {}
datamaps["id2item_dict"] = id2item_dict
datamaps["item2id_dict"] = item2id_dict

import json
json_str = json.dumps(datamaps)
with open(f"../{dataset_name}/datamaps.json", 'w') as out:
    out.write(json_str)
    

1002101

In [141]:
datamaps

{'id2item_dict': {3: "Love's Labour's Lost: Performed by Derek Jacobi, Geraldine McEwan & Cast",
  0: 'The Prophet',
  2: 'Othello: Complete & Unabridged',
  5: 'The Little Engine That Could',
  1: 'A Woman of Substance',
  4: "A Child's Garden of Verses",
  8: 'Green Eggs and Ham',
  7: 'Lion, the Witch and the Wardrobe',
  6: 'Go, Dog. Go! (Beginner Series)',
  10: "Magician's Nephew - Folio Society Hardcover",
  12: 'The Secret Garden',
  9: 'Fox in Socks (Dr.Seuss Board Books)',
  11: 'Princess and the Goblin (Abridged Classics)',
  14: 'PREY.',
  13: 'Cat in the Hat Song Book',
  15: 'Gilead',
  16: 'The Monk Who Sold His Ferrari: A Remarkable Story About Living Your Dreams',
  17: 'For Whom the Bell Tolls',
  18: 'The Sas Survival Handbook',
  19: 'The One Minute Manager',
  20: 'The Winds of War',
  21: 'North and South',
  23: 'Without Remorse',
  22: 'Kiss The Girls',
  26: 'A Feast for Crows (Song of Ice and Fire)',
  24: 'The Golden Fool (Tawny Man)',
  25: 'A Dance with Dra

In [142]:
item2pop_dict = item_pop.set_index("itemid")["pop"].to_dict()
for i in id2item_dict.keys():
    if i not in item2pop_dict.keys():
        item2pop_dict[i] = 0
item2pop_dict

json_str = json.dumps(item2pop_dict)
with open(f"../{dataset_name}/LLM/popularity_datamaps.json", 'w') as out:
    out.write(json_str)

{3164: 43434,
 180: 38971,
 1349: 35158,
 2222: 35072,
 1729: 31347,
 1103: 30145,
 4123: 28117,
 232: 25095,
 3557: 20792,
 9680: 20769,
 1513: 19668,
 615: 18895,
 9304: 18439,
 4025: 17735,
 9771: 17391,
 9749: 17369,
 9718: 17362,
 9710: 17360,
 8337: 17345,
 68: 17255,
 2771: 16915,
 2886: 16245,
 2326: 16203,
 3955: 16084,
 3313: 15576,
 1953: 15399,
 3146: 14870,
 6336: 14501,
 3358: 14344,
 2620: 13866,
 675: 13859,
 3568: 13842,
 5365: 13318,
 7041: 12606,
 174: 12583,
 2435: 12492,
 3023: 12366,
 1358: 12304,
 4061: 12292,
 5637: 12187,
 7361: 11989,
 1097: 11965,
 5216: 11938,
 1540: 11888,
 8233: 11650,
 6033: 11505,
 2772: 11301,
 1494: 11261,
 1855: 11074,
 2324: 10972,
 9799: 10966,
 966: 10848,
 3094: 10785,
 3509: 10761,
 1989: 10695,
 9670: 10473,
 6310: 10452,
 3776: 10315,
 188: 10169,
 1321: 10050,
 1308: 10037,
 168: 10035,
 1469: 10000,
 88: 9892,
 8245: 9682,
 5263: 9469,
 1861: 9405,
 9677: 9391,
 1272: 9337,
 1290: 9288,
 4587: 9242,
 363: 9056,
 5266: 9055,
 

129442

## Example of Loading Final Evaluation Data 

In [143]:
read_example_df = pd.read_csv(f"./Dataset/{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "itemid_history", "pos_target_index", "itemid_candidate", "pair_itemid_candidate", "pair_pos_target_index"]]
for col in ['itemid_history', 'itemid_candidate', "pair_itemid_candidate", "pair_pos_target_index"]:
    read_example_df[col] = read_example_df[col].apply(lambda x: eval(x))
    
read_example_df

FileNotFoundError: [Errno 2] No such file or directory: './Dataset//Book/LLM/topk_candidate@5_history@5.csv'

In [ ]:
read_example_df = pd.read_csv(f"./Dataset/{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "pos_target_index", "itemid_candidate"]]
read_example_df["itemid_candidate"] = read_example_df["itemid_candidate"].apply(lambda x: x[1:-1])
candidate = read_example_df['itemid_candidate'].str.split(',', expand=True)
for col in candidate.columns:
    candidate[col] = candidate[col].apply(lambda x: int(x))
candidate.values

array([[4020, 2033, 3152, 8805,  113],
       [2769, 5957, 1958, 6116, 2768],
       [5548, 8912, 6000, 5207, 4549],
       ...,
       [9055, 2614, 2878, 2237, 2617],
       [  55, 2703,  191, 2233, 1513],
       [4850, 8556, 4648, 1105, 6980]])